In [2]:
import numpy as np

# We are doing anytime computation of state-space modeling of a quadcopter with 3 degrees of freedom

# We first describe the state and the input
# X = [theta phi psi theta' phi' psi']
# U = [U1 U2 U3 U4]

# We have the initial input
X0 = Polyhedron(vertices = [(0,30,45,0.2,0.2,0.3)])
X0.vertices()

(A vertex at (0.0, 30.0, 45.0, 0.2, 0.2, 0.3),)

In [3]:
U0 = Polyhedron(vertices=[(3,4,2,3),(2,2,1,1)])
U0.vertices()

(A vertex at (2, 2, 1, 1), A vertex at (3, 4, 2, 3))

In [4]:
# We put some values before writing the matrices A and B

# distance from center to each motor
d = 0.2
# constant c
c = 5
# Torques along x,y and z axis
lx, ly, lz = 3, 3, 3

# We write the matrices as follows,
A = Matrix([[0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0]])

B = Matrix([[0,0,0,0],
            [0,0,0,0],
            [0,0,0,0],
            [0,d/lx,0,-d/lx],
            [d/ly,0,-d/ly,0],
            [-c/lz,c/lz,-c/lz,c/lz]])
C = np.hstack((A,B))
C

array([[ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.06666667,  0.        , -0.06666667],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.06666667,  0.        , -0.06666667,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -1.66666667,  1.66666667, -1.66666667,  1.66666667]])

In [5]:
P0 = [np.hstack((x,u)) for x in X0.vertices() for u in U0.vertices()]
P0 = Polyhedron(P0)
P0.vertices()

(A vertex at (0.0, 30.0, 45.0, 0.2, 0.2, 0.3, 2.0, 2.0, 1.0, 1.0),
 A vertex at (0.0, 30.0, 45.0, 0.2, 0.2, 0.3, 3.0, 4.0, 2.0, 3.0))

In [6]:
Xc = [np.matmul(C,p) for p in P0.vertices()]
Xc = Polyhedron(Xc)
Xc.vertices()

(A vertex at (0.2, 0.2, 0.3, 0.06666666667, 0.06666666667, 0.0),
 A vertex at (0.2, 0.2, 0.3, 0.06666666667, 0.06666666667, 3.333333333))

In [7]:
I = np.identity(6)
M = np.vstack((I,I))
M

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [8]:
Xs = [np.matmul(M,x) for x in Xc.vertices()]
Xs = Polyhedron(Xs)
Xs.vertices()

(A vertex at (0.2, 0.2, 0.3, 0.06666666667, 0.06666666667, 0.0, 0.2, 0.2, 0.3, 0.06666666667, 0.06666666667, 0.0),
 A vertex at (0.2, 0.2, 0.3, 0.06666666667, 0.06666666667, 3.333333333, 0.2, 0.2, 0.3, 0.06666666667, 0.06666666667, 3.333333333))

In [9]:
import itertools as it

def box(ep, n):
    s = [ep, -ep]
    y = it.product(s, repeat=n)
    z = list(y)
    return z

# l = box(2, 12)
# for v in l:
#     print(v)
ep = 2
n = 6
b = box(ep,n)
print(b)
zeros = tuple([0 for i in range(n)])
print(zeros)

for i in range(len(b)):
    b[i] = zeros + b[i]
print(b)

Box = Polyhedron(b)
Box.vertices()

[(2, 2, 2, 2, 2, 2), (2, 2, 2, 2, 2, -2), (2, 2, 2, 2, -2, 2), (2, 2, 2, 2, -2, -2), (2, 2, 2, -2, 2, 2), (2, 2, 2, -2, 2, -2), (2, 2, 2, -2, -2, 2), (2, 2, 2, -2, -2, -2), (2, 2, -2, 2, 2, 2), (2, 2, -2, 2, 2, -2), (2, 2, -2, 2, -2, 2), (2, 2, -2, 2, -2, -2), (2, 2, -2, -2, 2, 2), (2, 2, -2, -2, 2, -2), (2, 2, -2, -2, -2, 2), (2, 2, -2, -2, -2, -2), (2, -2, 2, 2, 2, 2), (2, -2, 2, 2, 2, -2), (2, -2, 2, 2, -2, 2), (2, -2, 2, 2, -2, -2), (2, -2, 2, -2, 2, 2), (2, -2, 2, -2, 2, -2), (2, -2, 2, -2, -2, 2), (2, -2, 2, -2, -2, -2), (2, -2, -2, 2, 2, 2), (2, -2, -2, 2, 2, -2), (2, -2, -2, 2, -2, 2), (2, -2, -2, 2, -2, -2), (2, -2, -2, -2, 2, 2), (2, -2, -2, -2, 2, -2), (2, -2, -2, -2, -2, 2), (2, -2, -2, -2, -2, -2), (-2, 2, 2, 2, 2, 2), (-2, 2, 2, 2, 2, -2), (-2, 2, 2, 2, -2, 2), (-2, 2, 2, 2, -2, -2), (-2, 2, 2, -2, 2, 2), (-2, 2, 2, -2, 2, -2), (-2, 2, 2, -2, -2, 2), (-2, 2, 2, -2, -2, -2), (-2, 2, -2, 2, 2, 2), (-2, 2, -2, 2, 2, -2), (-2, 2, -2, 2, -2, 2), (-2, 2, -2, 2, -2, -2), (-2, 2,

(A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, -2, -2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, -2, -2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, -2, 2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, -2, 2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, 2, -2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, 2, -2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, 2, 2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, -2, 2, 2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, -2, -2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, -2, -2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, -2, 2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, -2, 2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, 2, -2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, 2, -2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, 2, 2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, -2, 2, 2, 2, 2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, 2, -2, -2, -2, -2),
 A vertex at (0, 0, 0, 0, 0, 0, -2, 2, -2, -2, -2, 2),
 A vertex

In [16]:
Xc = Xs + Box
Xc.inequalities()

(An inequality (0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0) x + 2.0 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0) x + 0.0 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0) x + 1.8 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0) x + 1.8 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0) x + 1.7 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0) x + 1.933333333 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0) x + 1.933333333 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0) x + 2.0 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0) x + 2.066666667 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0) x + 2.066666667 >= 0,
 An inequality (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.

In [27]:
A = Polyhedron(eqns = [[-1,1,0],[-2,0,1]])
A.vertices()

(A vertex at (1, 2),)